<a href="https://colab.research.google.com/github/DerManjuel/MDL/blob/Manuel/MDL_Exercise5_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Deep Learning
## Exercise 5: Model Distillation & Ternary Nets

The goal of this exercise is to implement methods that allow to compress deep learning models via model distillation and ternary weights. This enables the use of deep learning in medicine due to its real-time ability and implementation on weaker mobile devices.

In [ ]:
#run pip install for pytorch flop counter before first use
!pip install onnx wget
!pip install --upgrade git+https://github.com/Lyken17/pytorch-OpCounter.git

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import time
import shutil,gzip
import wget

#some functions to count unique parameters and sparsity are provided
def countParameters(net):
    model_parameters = filter(lambda p: p.requires_grad, net.parameters())
    params = sum([p.numel() for p in model_parameters])
    return params

def countUnique(net):
    unique = 0
    for m in net.modules():
        if(isinstance(m,nn.Conv2d)):
            unique += len(np.unique(m.weight.data.cpu().flatten().numpy()))
    return unique
    #print('#unique',unique)

def countSparsity(net):
    count_nonzero = 0; count_zero = 0
    for m in net.modules():
        if(isinstance(m, nn.Conv2d)):
            count_nonzero += torch.sum((m.weight.data!=0).float())
            count_zero += torch.sum((m.weight.data==0).float())
    return count_zero/(count_zero+count_nonzero)


## Dataset
We will use the data of the Patch Camelyon (tupac16) Challenge. It consists of $327\,680$ color images extracted from histopathologic scans of lymph node sections. The task is to classify the presence of metastatic tissue (global binary labels are given). The images were preprocessed to a spatial dimension of $48\times48$ and split to 65k for training and 16k for testing images. See [here](https://www.kaggle.com/competitions/histopathologic-cancer-detection/overview) for further details.

In [ ]:
#loading the patch-based wholeslide histopathology data (uint8) and converting it to torch tensors
import os

dataset_url = 'https://cloud.imi.uni-luebeck.de/s/sjjiReHqSnokJ2n/download'

def get_data(data_url):
    filename = './patchCamelyon8c.mat'
    if not os.path.exists(filename):
        filename = wget.download(data_url)

get_data(dataset_url)


data = scipy.io.loadmat('patchCamelyon8c.mat')

img_train = torch.from_numpy(data['img_train'].astype('float32')/255)
img_test = torch.from_numpy(data['img_test'].astype('float32')/255)

label_train = torch.from_numpy(data['label_train']).long()
label_test = torch.from_numpy(data['label_test']).long()

Let's visualize an example for the two classes. You can run the cell multiple times, getting each time new random examples.

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2)
idx_negative = torch.where(label_train.squeeze() == 0)[0]
idx_negative = idx_negative[torch.randperm(len(idx_negative))[0]]

idx_positive = torch.where(label_train.squeeze() == 1)[0]
idx_positive = idx_positive[torch.randperm(len(idx_positive))[0]]

print('index for no metastatic tissue:', idx_negative.item())
print('index for metastatic tissue:', idx_positive.item())

ax1.imshow(img_train[idx_negative].permute(1, 2, 0))
ax1.set_title('no metastatic tissue')
ax2.imshow(img_train[idx_positive].permute(1, 2, 0))
ax2.set_title('metastatic tissue')
plt.show()

We load a pretrained `VGG11`, inspect its architecture and gather some computational information.

In [ ]:
from torchinfo import summary
net = torchvision.models.vgg11_bn(pretrained='True')

summary(net, (128, 3, 48, 48))

## Task 0 (15 points): Modify a pretrained VGG11_BN network for the given training data
Complete the function below, which returns a VGG11-Net with its architecture modified accordingly to match the tupac16 dataset.
+ Replace the layer `net.avgpool` with an adaptive average pool of output size $1\times1$
+ Create a new classifier as `nn.Sequential` with two linear layers ($512\times256$ and $256\times2$) including one ReLU and no batch-norm.

In [ ]:
def tupac16_vgg11():
    net = torchvision.models.vgg11_bn(pretrained='True')
    net.avgpool = # todo
    net.classifier = # todo

    return net

Check the reduced parameter count using `summary` from `torchinfo`. Your should obtain about $9\,357\,826$ parameters.

In [ ]:
# todo

## Task1 (25 points): Fine tuning
Fine-tune this network for 16 sub-epochs on the tupac16 dataset. One sub-epoch is defined as a random quarter of the training pathes. Use `torch.randperm` to generate the needed indices for every epoch. The batch size should be 128. Choose Adam as an optimizer with an initial learning rate of 0.0005 and an exponential learning rate scheduler with `gamma=0.9`. After training, evaluate the model on the test data. It should yield an accuracy about $94\%$.

**Note:** Task 2 and 3 can be performed independently, but you should store each trained network under a new filename (for comparisons). In the following all techniques should only be applied to the feature-part of the network (and not the classifier layers).

**Hint**: If you struggle with implementing of the training routine, have a look at the previous exercises.

In [ ]:
# Task1 - Training loop

# todo

## Task 2 (60 points): Network Pruning through increased Sparsity
+ Start with the same modified, pre-trained (not fine-tuned) vgg11_bn as before by calling your `tupac16_vgg11` method.
+ Reuse the training routine from above.
+ Add a sparsity promoting L1-loss (sum of absolute values) with a weight factor of 0.04 to the classification loss on the weights and bias of each BatchNorm2d.
    + Therefor iterate over all modules of the net using `modules()` and determine the layer type using `isinstance()`
+ Retrain the network.
+ Evaluate its test accuracy (will drop slightly to ~89%).

In [ ]:
# Task 2

# todo

Write a function that determines a threshold for input/output neurons to be set to zero (the ones which have been reduced in absolute value using the sparsity constraint). You can use the function `topk`, which outputs both the values and indices sorted around a chosen quantile/percentile. Here we simply use the median to set 50% of values to zero.

When applied correctly (as incoming & outgoing mask) for each Conv2d layer, it reduces the nonzero parameters by ~75% (the first incoming & last outgoing Conv2d are not masked).
Note that BatchNorm has four tensors and two index masks have to be applied as follows:

`B = A[idx_next,:,:,:][:,idx_prev,:,:]`

Now you can replace all Conv2d and BatchNorm2d layers with smaller filters (and copy their weights) so that we have the following sequence of channels: 3, 32, 64, (2x)128, (3x)256, 512.
Evaluate the slimmed network (you could observe a slight improvement to ~92%) and confirm that the required computations are reduced to 12 GFlops.

In [ ]:
# task 2 Network slimming (construct lean filters)

sparsity_s = 0.04
ternary = False

model_name = 'tupac_sparse'

net = torch.load(model_name+'_net.pth')

net.eval()
net.cuda()

mask = torch.ones(3).cuda()
idx_prev = torch.arange(3).long().cuda()
q75 = 0.5
print('#params before',countParameters(net.features))
print('#features sparsity',countSparsity(net.features))


for c in range(len(net.features)-2):
    if isinstance(net.features[c], nn.Conv2d):
        if(c==25):
            q75=1
        # todo


# evaluation for task 2

print('#params after',countParameters(net.features))
print('#features sparsity',countSparsity(net.features))

idx_epoch = torch.arange(16384).view(128,-1)
val_acc = 0

for iter in range(idx_epoch.size(1)):
    idx_iter = idx_epoch[:,iter]

    with torch.no_grad():
        #forward path and loss
        outputs = net(img_test[idx_iter,:,:,:].cuda())
    val_acc += torch.sum((outputs.argmax(1).cpu()==label_test[0,idx_iter]).float())/16384.0

print(val_acc)

## Bonus Task - Ternary Nets:
Start with the same modified, pre-trained (not fine-tuned) vgg11_bn as before. Finally, we want to explore, how the memory storage can be reduced with little loss. Here a ternary weight approximation will be used for which we first estimate a per-channel $\Delta$ for each weight in Conv2d given the rule-of-thumb below.

$\begin{align}
    \Delta &= \frac{0.7}{n}\sum^n_{i=1}|W_i|\\
    \tilde{W}_i&=
    \begin{cases}
        +1, &\text{ if } W_i > \Delta\\
        0,  &\text{ if } |W_i| \leq \Delta\\
        -1,  &\text{ else }
    \end{cases}\\
    n_\Delta &= \sum_i|\tilde{W}_i|\\
    \alpha &= \frac{1}{n_\Delta}\sum_i|\tilde{W}_i||W_i|
\end{align}$

Tip: after calculating the absolute values the mean has to be computed over all but the 0-th dimension.
The obtained ternary weights have lost their magnitude, therefore the parameter $\alpha$ (again per-channel) is computed and multiplied with the weight tensor.

Test your function with the check implemented below. For a $128\times64\times3\times3$ kernel the number of unique entries is reduced from more than 70 thousand to just 257 ($2 \cdot 128 + 1$).

In [ ]:
# template for function in bonus task
def approx_weights(w_in,flag=True):
    if(flag):
        with torch.no_grad():
            a,b,c,d = w_in.size()
            delta = # todo
            alpha = # todo
            w_out = # todo
    else:
        w_out = w_in
    return w_out

# check the number of unique values before/after ternary approximation
w_in = net.features[8].weight.clone().detach()
w_approx = approx_weights(w_in,True)
print('#unique',len(np.unique(w_in.data.cpu().flatten().numpy())))
print('#unique',len(np.unique(w_approx.data.cpu().flatten().numpy())))

To effectively train a network with weight quantisation, it is important to only use the ternary weights during forward/backward path, but update their gradients in full precision.

Implement a loop that stores full precision weights in a list of tensors and replaces the `.data` values with their approximation just before calling the forward pass (and zero_grad).
Reassign these backup copies after `loss.backward()` and `before optimizer.step()`. Retrain your network and take care to perform the weight quantisation the same way during test evaluation. The test accuracy should be around 85-90% during the epochs.

**Tip:** you could use `.pop(0)` to (iteratively) access and remove the first object of a list.

In [ ]:
# todo